In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pandas_profiling as ppf
# from tensorboardX import SummaryWriter
from sklearn.model_selection import train_test_split
# writer=SummaryWriter(logdir='./logistic')
torch.__version__

'1.2.0+cu92'

## logistic回归介绍


logistic回归是一种广义线性回归（generalized linear model），与多重线性回归分析有很多相同之处。它们的模型形式基本上相同，都具有 wx + b，其中w和b是待求参数，其区别在于他们的因变量不同，多重线性回归直接将wx+b作为因变量，即y =wx+b,而logistic回归则通过函数L将wx+b对应一个隐状态p，p =L(wx+b),然后根据p 与1-p的大小决定因变量的值。如果L是logistic函数，就是logistic回归，如果L是多项式函数就是多项式回归。

说的更通俗一点，就是logistic回归会在线性回归后再加一层logistic函数的调用。

logistic回归主要是进行二分类预测，我们在激活函数时候讲到过 Sigmod函数，Sigmod函数是最常见的logistic函数，因为Sigmod函数的输出的是是对于0~1之间的概率值，当概率大于0.5预测为1，小于0.5预测为0。

以下为僵尸企业分类的逻辑回归实践

In [2]:
encoded_all_data=pd.read_csv(r'./data/created_data/encoded_all_data.csv').reset_index(drop=True).set_index('ID')
init_x=encoded_all_data.dropna(subset=['flag']).drop(['flag'],axis=1)
init_y=encoded_all_data.dropna(subset=['flag'])['flag']
# encoded_all_data

In [3]:
train_X,val_X, train_y, val_y = train_test_split(init_x,init_y,test_size=0.30, random_state=0)
display(train_X,train_y)
display(val_X,val_y)
val_X.dtypes

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,
5992282.0,-0.140242,-0.139879,-0.325154,-0.325293,-0.114495,-0.612358,0.269269,-0.340802,-0.285456,-0.285470,-0.693239,-0.705462
5982045.0,1.966176,1.966437,0.052762,-0.216089,1.596196,1.148906,0.533604,3.938261,-0.285456,-0.285470,2.712321,1.191353
895831.0,0.786722,0.787260,-0.325154,-0.325293,-0.716335,0.826067,0.255958,-0.463019,0.547068,0.546235,0.432168,0.348116
5985243.0,-0.322979,-0.322662,-0.325154,-0.214615,-0.155016,-0.640345,0.219881,-0.380627,-0.283383,-0.283391,-0.729194,-0.748672
5983883.0,-0.042745,-0.042454,-0.218925,-0.219351,-0.306733,-0.338618,0.228194,-0.386355,-0.285456,-0.190409,-0.401226,-0.508057
...,...,...,...,...,...,...,...,...,...,...,...,...
5984896.0,1.196130,1.196813,-0.325154,-0.325293,1.549562,0.811447,0.434845,1.086850,-0.285456,-0.285470,0.414652,0.372022
5996407.0,-0.376463,-0.376167,0.036678,0.037324,0.111706,-0.215827,0.511119,-0.046966,-0.062900,-0.062541,-0.365487,-0.666507
5994688.0,-0.376463,-0.376167,-0.132856,-0.133514,0.123423,-0.215391,-0.447947,0.100624,0.266530,0.267311,-0.224783,0.548445


ID
5992282.0    0.0
5982045.0    0.0
895831.0     1.0
5985243.0    0.0
5983883.0    0.0
            ... 
5984896.0    0.0
5996407.0    0.0
5994688.0    0.0
5985391.0    0.0
584259.0     1.0
Name: flag, Length: 24953, dtype: float64

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,
5997249.0,-0.034880,-0.034590,0.016378,0.015317,0.578049,-0.139479,-0.210866,0.160015,-0.285456,-0.285470,-0.058149,-0.227895
5999913.0,-0.331225,-0.330930,-0.325154,-0.325293,-0.158613,-0.652001,0.209697,-0.410833,-0.279166,-0.279187,-0.744123,-0.764257
2088184.0,-0.220277,-0.219916,-0.325154,-0.325293,-0.300022,-0.528869,0.043826,-0.463019,-0.112608,-0.112513,-0.558673,-0.436801
5980104.0,-0.358507,-0.358211,-0.322216,-0.322362,-0.150429,-0.640118,0.248568,-0.396280,-0.259479,-0.259422,-0.715331,-0.787033
3306270.0,-0.186045,-0.185752,-0.325154,-0.325293,-0.454422,-0.209520,-0.554775,-0.463019,-0.285456,-0.285470,-0.190211,0.273736
...,...,...,...,...,...,...,...,...,...,...,...,...
2668569.0,0.432479,0.432943,-0.008793,-0.009786,-0.815304,0.637272,-0.937810,-0.463019,-0.012347,-0.012578,0.678724,1.644362
2735249.0,-0.376463,-0.376167,-0.054913,-0.054466,-0.943644,0.502727,-0.324949,-0.463019,1.385249,1.388950,0.971290,0.615503
5978857.0,-0.376463,-0.376167,-0.325154,-0.325293,-0.110419,-0.363604,0.194189,-0.268188,0.223692,0.223322,-0.558967,-0.549562


ID
5997249.0    0.0
5999913.0    0.0
2088184.0    1.0
5980104.0    0.0
3306270.0    1.0
            ... 
2668569.0    1.0
2735249.0    1.0
5978857.0    0.0
1604747.0    1.0
5985633.0    0.0
Name: flag, Length: 10695, dtype: float64

zhaiquanrongzi_chengben                 float64
zhaiquanrongzi_edu                      float64
neiburongzi_and_maoyirongzi_chengben    float64
neiburongzi_and_maoyirongzi_edu         float64
jinglirun                               float64
lirunzonge                              float64
suoyouzhe_quanyiheji                    float64
nashui_zonge                            float64
guquanrognzi_chengben                   float64
guquanrognzi_edu                        float64
yingye_zongshouru                       float64
fuzhai_zonge                            float64
dtype: object

#### logistic regression model

In [4]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(12,2) # 由于16个维度已经固定了，所以这里写16
    def forward(self,x):
        x=self.fc(x)
        out=torch.sigmoid(x)
        return out


def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

In [5]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [6]:
device = get_default_device()
device

device(type='cuda')

In [7]:
# 网络模型实例化
net=LR()
to_device(net, device)
# 使用CrossEntropyLoss损失
criterion=nn.CrossEntropyLoss() # BCEloss待测
# 优化器
optm=torch.optim.Adam(net.parameters(),lr=0.1) 
# 训练1500次
epochs=1000

#### train model

In [10]:
for i in range(epochs):
#      指定模型为训练模式，计算梯度
    net.train()
#      输入值都需要转化成torch的Tensor
    x=to_device(torch.from_numpy(train_X.values).float(),device)
    y=to_device(torch.from_numpy(train_y.values).long(),device)
#     输入数据
    y_hat=net(x)
#     计算损失
    loss=criterion(y_hat,y) 
#     前一步的损失清零
    optm.zero_grad() 
#     反向传播 
    loss.backward() 
#     更新参数
    optm.step() 
#     writer.add_scalar('tra_loss',loss,i)

#     if True:
    if (i+1)%200 ==0 : # 这里我们每100次输出相关的信息
#         指定模型为计算模式
        net.eval()
        test_in=to_device(torch.from_numpy(val_X.values).float(),device)
        test_l=to_device(torch.from_numpy(val_y.values).long(),device)
        test_out=net(test_in)
#          使用我们的测试函数计算准确率
        accu=test(test_out,test_l)
#         writer.add_scalar('tes_loss',loss.item(),i)
#         writer.add_scalar('tes_acc',accu,i)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.4f}".format(i+1,loss.item(),accu))    

Epoch:200,Loss:0.3296,Accuracy：0.9820
Epoch:400,Loss:0.3296,Accuracy：0.9820
Epoch:600,Loss:0.3296,Accuracy：0.9820
Epoch:800,Loss:0.3296,Accuracy：0.9820
Epoch:1000,Loss:0.3295,Accuracy：0.9820
Epoch:1200,Loss:0.3295,Accuracy：0.9820
Epoch:1400,Loss:0.3295,Accuracy：0.9820
Epoch:1600,Loss:0.3295,Accuracy：0.9820
Epoch:1800,Loss:0.3294,Accuracy：0.9820
Epoch:2000,Loss:0.3294,Accuracy：0.9820
Epoch:2200,Loss:0.3294,Accuracy：0.9819
Epoch:2400,Loss:0.3294,Accuracy：0.9818
Epoch:2600,Loss:0.3294,Accuracy：0.9817
Epoch:2800,Loss:0.3293,Accuracy：0.9819
Epoch:3000,Loss:0.3293,Accuracy：0.9820
Epoch:3200,Loss:0.3293,Accuracy：0.9820
Epoch:3400,Loss:0.3293,Accuracy：0.9819
Epoch:3600,Loss:0.3293,Accuracy：0.9819
Epoch:3800,Loss:0.3293,Accuracy：0.9820
Epoch:4000,Loss:0.3292,Accuracy：0.9820
Epoch:4200,Loss:0.3292,Accuracy：0.9820
Epoch:4400,Loss:0.3292,Accuracy：0.9819
Epoch:4600,Loss:0.3292,Accuracy：0.9819
Epoch:4800,Loss:0.3292,Accuracy：0.9818
Epoch:5000,Loss:0.3292,Accuracy：0.9818
Epoch:5200,Loss:0.3292,Accura

In [11]:
# writer.close() 97.96%

#### 为无标签的数据标上flag

In [12]:
non_label=encoded_all_data.drop(init_x.index)
non_label_X=non_label.drop(['flag'],axis=1)
estimate_y=estimate_y=np.array([1]*non_label_X.shape[0])
non_label_X

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge
ID,,,,,,,,,,,,
1080523.0,0.153910,0.154316,0.334840,0.335179,-0.423658,-0.235871,-0.522061,-0.463019,-0.285456,-0.285470,-0.258610,0.416919
1080756.0,2.037227,2.037933,-0.188487,-0.189200,-0.737513,1.150920,-0.921308,-0.463019,-0.285456,-0.285470,0.495737,1.237598
1080951.0,-0.376463,-0.376167,0.396778,0.394059,-0.603652,0.351831,0.076741,-0.463019,-0.285456,-0.285470,0.175824,0.557640
1080972.0,-0.376463,-0.376167,-0.254944,-0.255188,-0.230101,-0.618374,0.206208,-0.463019,-0.263837,-0.263875,-0.726563,-0.773093
1081027.0,-0.376463,-0.376167,-0.325154,-0.325293,-0.415439,-0.323241,0.438912,-0.463019,0.185092,0.184636,-0.283236,-0.388421
...,...,...,...,...,...,...,...,...,...,...,...,...
5999614.0,1.952389,1.953307,-0.325154,-0.325293,0.250238,0.206426,0.982232,0.157481,-0.285456,-0.285470,0.070694,-0.035223
5999746.0,-0.376463,-0.376167,0.100278,0.098578,0.038375,-0.239628,-0.104054,0.203587,-0.042700,-0.042990,-0.275084,-0.262263
5999945.0,-0.376463,-0.376167,-0.302229,-0.213973,-0.191376,-0.644977,0.255412,-0.449279,-0.276388,-0.276409,-0.761829,-0.810960


In [13]:
net.eval()
test_in=to_device(torch.from_numpy(non_label_X.values).float(),device)
test_out=net(test_in)
new_label=test_out.max(-1)[1].tolist()
new_label

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [14]:
non_label_X['flag']=new_label
non_label_X

,zhaiquanrongzi_chengben,zhaiquanrongzi_edu,neiburongzi_and_maoyirongzi_chengben,neiburongzi_and_maoyirongzi_edu,jinglirun,lirunzonge,suoyouzhe_quanyiheji,nashui_zonge,guquanrognzi_chengben,guquanrognzi_edu,yingye_zongshouru,fuzhai_zonge,flag
ID,,,,,,,,,,,,,
1080523.0,0.153910,0.154316,0.334840,0.335179,-0.423658,-0.235871,-0.522061,-0.463019,-0.285456,-0.285470,-0.258610,0.416919,1
1080756.0,2.037227,2.037933,-0.188487,-0.189200,-0.737513,1.150920,-0.921308,-0.463019,-0.285456,-0.285470,0.495737,1.237598,1
1080951.0,-0.376463,-0.376167,0.396778,0.394059,-0.603652,0.351831,0.076741,-0.463019,-0.285456,-0.285470,0.175824,0.557640,1
1080972.0,-0.376463,-0.376167,-0.254944,-0.255188,-0.230101,-0.618374,0.206208,-0.463019,-0.263837,-0.263875,-0.726563,-0.773093,1
1081027.0,-0.376463,-0.376167,-0.325154,-0.325293,-0.415439,-0.323241,0.438912,-0.463019,0.185092,0.184636,-0.283236,-0.388421,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999614.0,1.952389,1.953307,-0.325154,-0.325293,0.250238,0.206426,0.982232,0.157481,-0.285456,-0.285470,0.070694,-0.035223,0
5999746.0,-0.376463,-0.376167,0.100278,0.098578,0.038375,-0.239628,-0.104054,0.203587,-0.042700,-0.042990,-0.275084,-0.262263,0
5999945.0,-0.376463,-0.376167,-0.302229,-0.213973,-0.191376,-0.644977,0.255412,-0.449279,-0.276388,-0.276409,-0.761829,-0.810960,0


In [15]:
logistic_final=encoded_all_data.combine_first(non_label_X)
logistic_final.info()
logistic_final.to_csv(r'./data/created_data/logistic_final.csv')

<class 'pandas.core.frame.DataFrame'>
Float64Index: 45931 entries, 28.0 to 5999999.0
Data columns (total 13 columns):
flag                                    45931 non-null float64
fuzhai_zonge                            45931 non-null float64
guquanrognzi_chengben                   45931 non-null float64
guquanrognzi_edu                        45931 non-null float64
jinglirun                               45931 non-null float64
lirunzonge                              45931 non-null float64
nashui_zonge                            45931 non-null float64
neiburongzi_and_maoyirongzi_chengben    45931 non-null float64
neiburongzi_and_maoyirongzi_edu         45931 non-null float64
suoyouzhe_quanyiheji                    45931 non-null float64
yingye_zongshouru                       45931 non-null float64
zhaiquanrongzi_chengben                 45931 non-null float64
zhaiquanrongzi_edu                      45931 non-null float64
dtypes: float64(13)
memory usage: 4.9 MB


In [59]:
# ppf.ProfileReport(logistic_final)